# Import Dependencies

In [1]:
import mesa
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

# Resource Classes

In [2]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - contains an amount of sugar
    - grows one amount of sugar each turn
    '''
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(model, unique_id)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

In [3]:
class Spice(mesa.Agent):
    '''
    Spice:
    - contains an amount of spice
    - grows 1 amount of spice each turn
    '''
    def __init__(self):
        print('I am spice')

# Trader Class

In [4]:
class Trader(mesa.Agent):
    '''
    Trader:
    - has a metabolism for sugar and spice
    - harvests and trades sugar and spice to survive and thrive
    '''
    def __init__(self):
        print('I am a trader')

# Model Class

In [5]:
class SugarscapeG1mt(mesa.Model):
    '''
    A model class to manage Sugarscape with Traders (G1mt)
    from Growing Artificial Societies by Axtell and Epstein
    '''
    def __init__(self, width=50, height=50):
        # Indicate width and height of sugarscape
        self.width = width
        self.height = height

        # Instantiate mesa grid class
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

        # Read in landscape file from supplementary material
        sugar_distribution = np.genfromtxt('sugar-map.txt')
        spice_distribution = np.flip(sugar_distribution, 1) # Flip on x and y axis
        # print(sugar_distribution.shape)
        # print(sugar_distribution[30][15]) # Call specific cell (x,y)
        # plt.imshow(spice_distribution, origin='lower')

        agent_id = 0
        for _, x, y in self.grid.coord_iter():
            max_sugar = sugar_distribution[x,y]
            if max_sugar > 0:
                sugar = Sugar(agent_id, self, (x,y), max_sugar)
                self.grid.place_agent(sugar, (x,y))
                agent_id += 1

        for _, x, y in self.grid.coord_iter():
            print(_, (x, y))


In [7]:
model = SugarscapeG1mt()

[] (0, 0)
[] (0, 1)
[] (0, 2)
[] (0, 3)
[] (0, 4)
[] (0, 5)
[] (0, 6)
[] (0, 7)
[] (0, 8)
[] (0, 9)
[] (0, 10)
[] (0, 11)
[] (0, 12)
[] (0, 13)
[] (0, 14)
[] (0, 15)
[] (0, 16)
[] (0, 17)
[<__main__.Sugar object at 0x0000020CE9E03500>] (0, 18)
[<__main__.Sugar object at 0x0000020CE9E03C50>] (0, 19)
[<__main__.Sugar object at 0x0000020C92F2EEA0>] (0, 20)
[<__main__.Sugar object at 0x0000020C92F2EFC0>] (0, 21)
[<__main__.Sugar object at 0x0000020C92F2C530>] (0, 22)
[<__main__.Sugar object at 0x0000020C92E57BF0>] (0, 23)
[<__main__.Sugar object at 0x0000020C92ED2C30>] (0, 24)
[<__main__.Sugar object at 0x0000020C92E781A0>] (0, 25)
[<__main__.Sugar object at 0x0000020C92E7ACF0>] (0, 26)
[<__main__.Sugar object at 0x0000020C92E78230>] (0, 27)
[<__main__.Sugar object at 0x0000020C92F86150>] (0, 28)
[<__main__.Sugar object at 0x0000020C92F86180>] (0, 29)
[<__main__.Sugar object at 0x0000020C92F860F0>] (0, 30)
[<__main__.Sugar object at 0x0000020C92F86120>] (0, 31)
[<__main__.Sugar object at 0